In [1]:
import pickle
import os
import numpy as np
# google 的 NN coding 套件
import tensorflow as tf
import imgaug as ia
from imgaug import augmenters as iaa
import random
import cv2
import matplotlib.pyplot as plt
import pandas

c:\users\mmnet\appdata\local\programs\python\python35\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
'''setting'''
gpus = [1] # Here I set CUDA to only see one GPU
os.environ['CUDA_VISIBLE_DEVICES']=','.join([str(i) for i in gpus])

In [3]:
model_dir = './reg/'
data_dir = './dataset/'
logs_dir = model_dir + 'logs/'
MAX_ITERATION = int(1e5 + 1)
training = False

In [4]:
db_helen = pickle.load(open( data_dir+"HELEN.pickle", "rb" ) )
print(db_helen.keys())
#db_300W['img'].keys()

dict_keys(['img', 'pts'])


In [5]:
def batch_norm(x, train_phase, name='bn_layer'):
    #with tf.variable_scope(name) as scope:
    batch_norm = tf.layers.batch_normalization(
            inputs=x,
            momentum=0.9, epsilon=1e-5,
            center=True, scale=True,
            training = train_phase,
            name=name
    )
    return batch_norm

def conv_blk (inputs,n_filter, train_phase, name = 'conv_blk'):
    with tf.variable_scope(name):
        c1 = tf.layers.conv2d(inputs, filters=n_filter[0], kernel_size=[3,3], strides=(1,1), padding='same')       
        c1_bn = batch_norm(c1, train_phase, name='c1_bn')
        c1_relu = tf.nn.relu(c1_bn)
        c2 = tf.layers.conv2d(c1_relu,filters=n_filter[1],kernel_size=[3,3],strides=(1,1),padding='same')        
        c2_bn = batch_norm(c2, train_phase, name='c2_bn')
        c2_relu = tf.nn.relu(c2_bn)
        return c2_relu

In [6]:
# Define Model Input (x) and Output (y_),  y_ = f(x)
x = tf.placeholder(tf.float32, [None, 224,224,3])
y_ = tf.placeholder(tf.float32, [None,68,2]) # 136
train_phase = tf.placeholder(tf.bool, name='phase_train')

y_one = tf.layers.flatten(y_)

# convolutional part
r1 = tf.layers.max_pooling2d(x,pool_size=[2,2],strides=(2,2))

h1 = conv_blk(r1, [64,64], train_phase, name='conv_blk1')
m1 = tf.layers.max_pooling2d(h1,pool_size=[2,2],strides=(2,2))

h2 = conv_blk(m1, [128,128], train_phase, name='conv_blk2')
m2 = tf.layers.max_pooling2d(h2,pool_size=[2,2],strides=(2,2))

h3 = conv_blk(m2, [256,256], train_phase, name='conv_blk3')
m3 = tf.layers.max_pooling2d(h3,pool_size=[2,2],strides=(2,2))

h4 = conv_blk(m3, [512,512], train_phase, name='conv_blk4')
m4 = tf.layers.max_pooling2d(h4,pool_size=[2,2],strides=(2,2))

flt = tf.layers.flatten(m4)

# fully connected part

f1_do = tf.layers.dropout(flt,rate=0.5)

f1 = tf.layers.dense(f1_do,256,activation=None)
f1_bn = batch_norm(f1, train_phase, name='f1_bn')
f1_relu = tf.nn.relu(f1_bn)

f2 = tf.layers.dense(f1_relu,136,activation=None)

y_out = tf.reshape(f2, shape = [-1,68,2])

In [7]:
total_parameters = 0
for variable in tf.trainable_variables():
    # shape is an array of tf.Dimension
    shape = variable.get_shape()
#         print(shape)
#         print(len(shape))
    variable_parameters = 1
    for dim in shape:
#             print(dim)
        variable_parameters *= dim.value
#         print(variable_parameters)
    total_parameters += variable_parameters
print('total_parameters', total_parameters)

total_parameters 11147464


In [8]:
# Define the Model Loss (4)
avg_losses = tf.reduce_mean(tf.reduce_mean(tf.sqrt(tf.reduce_sum(tf.squared_difference(y_, y_out),-1)),-1))

# Define the Optimizer (5)
train_step = tf.train.AdamOptimizer(0.01).minimize(avg_losses)

# y_pred = tf.argmax(tf.nn.softmax(y), 1, output_type=tf.int32)

# # Accuracy of the Model
# correct_prediction = tf.equal(y_pred, y_)
# accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# initialize the model
extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

print("Setting up Saver...")
saver = tf.train.Saver(tf.global_variables())

Setting up Saver...


In [9]:
sess = tf.InteractiveSession(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=False))
if (training == False):
    ckpt = tf.train.get_checkpoint_state(logs_dir)
    if ckpt and ckpt.model_checkpoint_path:
        saver.restore(sess, ckpt.model_checkpoint_path)
        print('Loading sucessfully')
    else:
        print('No checkpoint file found')
        raise
else:
    init = tf.global_variables_initializer()
    sess.run(init)

INFO:tensorflow:Restoring parameters from ./reg/logs/model.ckpt-76000
Loading sucessfully


In [10]:
# data iterator
def get_batch(X, Y, batch_size = 32):
    # print ('shuffle training dataset')
    idx = np.arange(len(X))    
    while True:
        np.random.shuffle(idx)
        tb = int(len(X)/batch_size)
        #print('total batches %d' % tb)
        for b_idx in range(tb):
            tar_idx = idx[(b_idx*batch_size):((b_idx+1)*batch_size)]
            t_batch_x = X[tar_idx]
            t_batch_y = Y[tar_idx]
            # print(b_idx, t_batch_x.shape, t_batch_y.shape)
            yield t_batch_x, t_batch_y

def data_augmentation(images, pts, rot=(-30, 30), s=(0.6, 1.0)):
    keypoints_on_images = []
    for idx_img in range(images.shape[0]):
        image = images[idx_img]
        height, width = image.shape[0:2]
        keypoints = []
        for p in range(pts.shape[1]):
            keypoints.append(ia.Keypoint(x=pts[idx_img,p,0], y=pts[idx_img,p,1]))
        keypoints_on_images.append(ia.KeypointsOnImage(keypoints, shape=image.shape))

    seq = iaa.Sequential([iaa.Affine(rotate=rot,scale=s)])
    seq_det = seq.to_deterministic() # call this for each batch again, NOT only once at the start

    # augment keypoints and images
    images_aug = seq_det.augment_images(images)
    keypoints_aug = seq_det.augment_keypoints(keypoints_on_images)
    
    pts_aug=[]
    for img_idx, keypoints_after in enumerate(keypoints_aug):
        img_pts_aug=[]
        for kp_idx, keypoint in enumerate(keypoints_after.keypoints):
            img_pts_aug.append([round(keypoint.x),round(keypoint.y)])
        pts_aug.append(np.asarray(img_pts_aug))

    pts_aug = np.asarray(pts_aug).astype(np.int32)
    
#     print('images_aug', images_aug.shape)
#     print('pts_aug', pts_aug.shape)
    return images_aug, pts_aug
      
# write image to file
def write_result(batch_xs_valid, batch_pts, iter_num):
    b = random.randint(0, batch_pts.shape[0]-1)
    img = batch_xs_valid[b].copy()
    pts = batch_pts[b] #print(pts)
    for p in range(pts.shape[0]):
        #print("p",p, pts[p+1,0],pts[p+1,1])
        cv2.circle(img,(pts[p,0],pts[p,1]), 2, (255,0,0), -1)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

    cv2.imwrite(model_dir + 'imgs/infer_'+str(iter_num)+'.png', img)
    
def eval_norm_error_image(infer, gt):
    # loss of all landmarks
    l2d = np.sum(np.sqrt(np.sum(np.square(infer-gt),axis=2)), axis=1)
    # distance of eye corners
    cd = np.sqrt(np.sum(np.square(gt[:,45,:]-gt[:,36,:]),axis=1))
    norm_error_image = l2d/cd/68
    return norm_error_image

In [11]:
if training == True:
    batches = get_batch(db_helen['img']['trainset'], db_helen['pts']['trainset'], batch_size = 32)
    # Train Model for 1000 steps
    hist_train_acc = []
    hist_valid_acc = []
    max_validloss = 99999
    for step in range(MAX_ITERATION):
        batch_xs, batch_ys = next(batches)
        batch_xs_aug, batch_ys_aug =data_augmentation(batch_xs, batch_ys)

        sess.run([extra_update_ops,train_step], feed_dict={x: batch_xs_aug, y_: batch_ys_aug, train_phase: True})

        if (step % 500 == 0):
            # get training accr
            idx = np.arange(len(db_helen['img']['trainset']))    
            tb = int(len(db_helen['img']['trainset'])/32)
            acc_train= []
            for b_idx in range(tb):
                tar_idx = idx[(b_idx*32):((b_idx+1)*32)]
                t_batch_x = db_helen['img']['trainset'][tar_idx]
                t_batch_y = db_helen['pts']['trainset'][tar_idx]
            acc_train.append(sess.run(avg_losses, feed_dict={x: t_batch_x, y_: t_batch_y, train_phase: False}))
            print("[T] Step: %d, loss:%g" % (step, np.mean(acc_train)))

            #tar_idx = idx[(b_idx*32):((b_idx+1)*32)]
            t_batch_x = db_helen['img']['testset']#[tar_idx]
            t_batch_y = db_helen['pts']['testset']#[tar_idx]
            infered_pts, acc_valid= sess.run([tf.reshape(y_out,shape=(-1,68,2)), avg_losses], feed_dict={x: t_batch_x,
                                                                                                     y_: t_batch_y,
                                                                                                     train_phase: False})
    #             acc_valid.append(valid_loss)
            write_result(db_helen['img']['testset'][np.arange(infered_pts.shape[0])], infered_pts, step)

            if np.mean(acc_valid) < max_validloss:
                saver.save(sess, logs_dir + "model.ckpt", step)
                print("[V*] Step: %d, loss:%g" % (step, np.mean(acc_valid)))
                max_validloss = np.mean(acc_valid)
            else:
                print("[V] Step: %d, loss:%g" % (step, np.mean(acc_valid)))

            hist_train_acc.append(np.mean(acc_train))
            hist_valid_acc.append(np.mean(acc_valid))
else: # evaluate
    t_batch_x = db_helen['img']['testset']#[tar_idx]
    t_batch_y = db_helen['pts']['testset']#[tar_idx]
    infered_pts, acc_valid= sess.run([tf.reshape(y_out,shape=(-1,68,2)), avg_losses], feed_dict={x: t_batch_x, y_: t_batch_y, train_phase: False})
    norm_error_image = eval_norm_error_image(infered_pts, t_batch_y)
    pandas.DataFrame({'loss':norm_error_image}).to_csv(model_dir + 'norm_error_image.csv')

In [12]:
sess.close()